In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Uploading Dataset

In [45]:
df = pd.read_csv('/content/org-diabetic_data.csv')

In [ ]:
df.shape

(101766, 50)

In [ ]:
df.head(10)

In [ ]:
df.info()

Univariate Analysis

In [ ]:
int_cols = df.select_dtypes(include='int64')

display(int_cols.describe())

In [ ]:
int_viz = df.select_dtypes(include='int64')
int_viz.hist(figsize=(10,10))

In [ ]:
categorical_cols = df.select_dtypes(include='object')

for col in categorical_cols.columns:
    print(f"Column: {col}")
    print(df[col].value_counts())
    print("-" * 30)

In [ ]:
# Visualize the frequency of each category
for col in categorical_cols.columns:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, y=col, order=df[col].value_counts().index, palette='viridis', hue=col, legend=False)
    plt.title(f'Frequency of {col}')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

Bivariate Analysis

In [ ]:
# relation bw
plt.figure(figsize=(8, 6))
sns.boxplot(x='readmitted', y='time_in_hospital', data=df)
plt.title('Time in Hospital by Readmission Status')
plt.xlabel('Readmitted')
plt.ylabel('Time in Hospital')
plt.show()

In [ ]:
# filter out the 'Not Mapped' race category
df_filtered = df[df['race'] != '?']

# grouping by columns - race and readmitted to count the occurrences
race_readmitted_counts = df_filtered.groupby(['race', 'readmitted']).size().unstack(fill_value=0)

# total count for each race
race_readmitted_counts['Total'] = race_readmitted_counts.sum(axis=1)

# readmission rates for each race
race_readmitted_counts['Readmission Rate (%)'] = (race_readmitted_counts['>30'] + race_readmitted_counts['<30']) / race_readmitted_counts['Total'] * 100

#plot
race_readmitted_counts[['NO', '<30', '>30']].plot(kind='bar', stacked=True, figsize=(10, 7))
plt.title('Readmission Status by Race')
plt.xlabel('Race')
plt.ylabel('Number of Patients')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Readmitted Status', loc='upper left')
plt.tight_layout()
plt.show()

#table
print("Readmission Rates by Race:")
print(race_readmitted_counts[['Total', '<30', '>30', 'Readmission Rate (%)']].sort_values(by='Readmission Rate (%)', ascending=False))


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='time_in_hospital', y='num_medications', data=df)
plt.title('Number of Medications vs. Time in Hospital')
plt.xlabel('Time in Hospital')
plt.ylabel('Number of Medications')
plt.show()

# corr coeff
correlation = df['time_in_hospital'].corr(df['num_medications'])
print(f"Correlation between Time in Hospital and Number of Medications: {correlation:.2f}")

In [ ]:
# grouping by columns - age and readmitted to count the occurrences
age_readmitted_counts = df.groupby(['age', 'readmitted']).size().unstack(fill_value=0)

# total count for each age group
age_readmitted_counts['Total'] = age_readmitted_counts.sum(axis=1)

# readmission rates for each age group
age_readmitted_counts['Readmission Rate (%)'] = (age_readmitted_counts['<30'] + age_readmitted_counts['>30']) / age_readmitted_counts['Total'] * 100

age_readmitted_counts = age_readmitted_counts.sort_index()

print("Readmission Rates by Age Group:")
print((age_readmitted_counts[['Total', '<30', '>30', 'Readmission Rate (%)']]).sort_values(by='Readmission Rate (%)', ascending=False))

# **DATA CLEANING**

In [46]:
# Dropping unwanted columns
columns_to_drop = ['weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']
df1 = df.drop(columns=columns_to_drop)

display(df1.shape)

(101766, 44)

In [47]:
#dropping columns that are near-zero variance predictors
col_drop = ['metformin-pioglitazone', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'chlorpropamide' , 'acetohexamide' ,  'troglitazone' , 'examide' , 'citoglipton' , 'tolbutamide']
df = df1.drop(columns=col_drop)
df.shape

(101766, 35)

Mapping IDs as per dataset

In [49]:
# Map column admission_type_id
df_r = df.replace({'admission_type_id': {
    1: 'Emergency',
    2: 'Urgent',
    3: 'Elective',
    4: 'Newborn',
    5: 'NA',
    6: 'NULL',
    7: 'Trauma Center',
    8: 'Not Mapped'
}})


In [50]:
# Map column discharge_disposition_id
df_r1 = df_r.replace({'discharge_disposition_id': {
    1: 'Discharged to home',
    2: 'Discharged/transferred to another short term hospital',
    3: 'Discharged/transferred to SNF',
    4: 'Discharged/transferred to ICF',
    5: 'Discharged/transferred to another type of inpatient care institution',
    6: 'Discharged/transferred to home with home health service',
    7: 'Left AMA',
    8: 'Discharged/transferred to home under care of Home IV provider',
    9: 'Admitted as an inpatient to this hospital',
    10: 'Neonate discharged to another hospital for neonatal aftercare',
    11: 'Expired',
    12: 'Still patient or expected to return for outpatient services',
    13: 'Hospice / home',
    14: 'Hospice / medical facility',
    15: 'Discharged/transferred within this institution to Medicare approved swing bed',
    16: 'Discharged/transferred/referred another institution for outpatient services',
    17: 'Discharged/transferred/referred to this institution for outpatient services',
    18: 'NULL',
    19: 'Expired at home. Medicaid only, hospice.',
    20: 'Expired in a medical facility. Medicaid only, hospice.',
    21: 'Expired, place unknown. Medicaid only, hospice.',
    22: 'Discharged/transferred to another rehab fac including rehab units of a hospital .',
    23: 'Discharged/transferred to a long term care hospital.',
    24: 'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.',
    25: 'Not Mapped',
    26: 'Unknown/Invalid',
    27: 'Discharged/transferred to a federal health care facility.',
    28: 'Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital',
    29: 'Discharged/transferred to a Critical Access Hospital (CAH).',
    30: 'Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere'
}})




In [51]:
# Map column admission_source_id
df_r2 = df_r1.replace({'admission_source_id': {
    1: 'Physician Referral',
    2: 'Clinic Referral',
    3: 'HMO Referral',
    4: 'Transfer from a hospital',
    5: 'Transfer from a Skilled Nursing Facility (SNF)',
    6: 'Transfer from another health care facility',
    7: 'Emergency Room',
    8: 'Court/Law Enforcement',
    9: 'Not Available',
    10: 'Transfer from critial access hospital',
    11: 'Normal Delivery',
    12: 'Premature Delivery',
    13: 'Sick Baby',
    14: 'Extramural Birth',
    15: 'Not Available',
    17: 'NULL',
    18: 'Transfer From Another Home Health Agency',
    19: 'Readmission to Same Home Health Agency',
    20: 'Not Mapped',
    21: 'Unknown/Invalid',
    22: 'Transfer from hospital inpt/same fac reslt in a sep claim',
    23: 'Born inside this hospital',
    24: 'Born outside this hospital',
    25: 'Transfer from Ambulatory Surgery Center',
    26: 'Transfer from Hospice'
}})




In [ ]:
df['readmitted_target'] = df_r2['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

print(df['readmitted_target'].value_counts())

df.head(4)

In [55]:
# Converting all data types to object for One-Hot Encoding
df = df.astype('object')


In [ ]:
categorical_cols = df.select_dtypes(include='object').columns.tolist()

exclude_cols = ['encounter_id', 'patient_nbr']
categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

# One-Hot Encoding to all categorical col
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=False, dummy_na=False)

display(df_encoded.head())
print("Final encoded shape:", df_encoded.shape)


In [58]:
df.shape

(101766, 36)

Downloading dataset

In [28]:
from google.colab import files

# Save to CSV
df_encoded.to_csv('cleaned_diabetes_data.csv', index=False)

files.download('cleaned_diabetes_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>